In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
#Read main data
data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\data_values.xlsx',sheet_name='data_transpose')
rainfall_data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\rainfall_data.xlsx',sheet_name='Sheet1')

# Read data necessary for normalization
ind_type=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\indicator_type.xlsx',sheet_name='Sheet1')

In [3]:
#defining normalization function for (dam capicity)/(surface rwr)
def damcap_normalize(ser1,ser2):
    result=ser1*0
    length=ser1.size
    for i in range(length):
        if ser1[i]>=ser2[i]:
            result[i]=(ser1.max()-ser1[i])/(ser1.max()-ser2[i])
        else:
            result[i]=(ser1[i]-ser1.min())/(ser2[i]-ser1.min())
        if result[i]<0.01:
            result[i]=0.01
    return result  

#defining aggregation functions
def aggregate(df,a=0.5):
    size=len(df.columns)
    result=(df.product(axis=1)**(1/size))*a+df.mean(axis=1)*(1-a)
    return result

#defining weighted aggregation functions
def weighted_addminagg(dfi,dfw,a=0.5):
    weight_array=np.array(dfw)[0]
    dfmin=dfi.min(axis=1)
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    result=a*dfmin+(1-a)*weighted_sum
    return result

def weighted_sum(dfi,dfw):
    weight_array=np.array(dfw)[0]
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    return weighted_sum

def geomean(dfi):
    size=len(dfi.columns)
    weighted_geomean=dfi.pow(weight_array,axis='columns').product(axis='columns')
    return weighted_geomean

def weighted_geomean(dfi,dfw):
    weight_array=np.array(dfw)[0]
    weighted_geomean=dfi.pow(weight_array,axis='columns').product(axis='columns')
    return weighted_geomean

def weighted_addgeo(dfi,dfw,a=0.5):
    weight_array=np.array(dfw)[0]
    weighted_geomean=dfi.pow(weight_array,axis='columns').product(axis='columns')
    weighted_sum=dfi.multiply(weight_array,axis='columns').sum(axis=1)
    result= a*weighted_geomean+(1-a)*weighted_sum
    return result

In [4]:
#selecting rainfall data after 1366 (nearly 30 years)
rainfall_modified=rainfall_data.loc[(rainfall_data['syear']>1366) & (rainfall_data['syear']<1399)]

#making a dictionary of provinces and their respective data
pr=list(rainfall_modified.ostan.unique())
prdict={elem:pd.DataFrame() for elem in pr}
for key in prdict.keys():
    prdict[key]=rainfall_modified[:][rainfall_modified.ostan==key]
    
#making a dictionary consisting of province names as keys and annual precipitation as respective values
annualrain_dict={x:pd.DataFrame() for x in pr}
for key in annualrain_dict.keys():
    annualrain_dict[key]=prdict[key].groupby('syear').precnew.sum()

#calculating coefficients of variation
interannual_varicoef={x:pd.DataFrame() for x in pr}
for key in interannual_varicoef.keys():
    interannual_varicoef[key]=annualrain_dict[key].std()/annualrain_dict[key].mean()
monthly_varicoef={x:pd.DataFrame() for x in pr}
for key in monthly_varicoef.keys():
    monthly_varicoef[key]=prdict[key].precnew.std()/prdict[key].precnew.mean()
annualevap_varicoef={x:pd.DataFrame() for x in pr}

#converting monthly coefficient of variation to dataframe and sorting it based on original data
monthlyvaricoeff_df = pd.DataFrame(monthly_varicoef.items(),columns=['province', 'monthly_varicoeff'])
monthlyvaricoeff_df = monthlyvaricoeff_df.drop([31])
monthlyvaricoeff_df = monthlyvaricoeff_df.set_index('province')
monthlyvaricoeff_df = monthlyvaricoeff_df.reindex(index=data['province'])
monthlyvaricoeff_df = monthlyvaricoeff_df.reset_index()

#converting annual coefficient of variation to dataframe and sorting it based on original data
intanvaricoeff_df = pd.DataFrame(interannual_varicoef.items(),columns=['province', 'rain_coeff_variability'])
intanvaricoeff_df = intanvaricoeff_df.drop([31])
intanvaricoeff_df = intanvaricoeff_df.set_index('province')
intanvaricoeff_df = intanvaricoeff_df.reindex(index=data['province'])
intanvaricoeff_df = intanvaricoeff_df.reset_index()

# adding calculated coefficients of variation to the data
data['rain_coeff_variation']=intanvaricoeff_df['rain_coeff_variability']
data['monthly_varicoeff']= monthlyvaricoeff_df['monthly_varicoeff']

In [5]:
#Creating a dataframe consisting of necessary variables for the calculation of sub-indicators
variables=pd.DataFrame()
variables['province']=data.province
variables['irwr']=data.precipitation-data.evaporation
variables['surf_irwr']=(data.precipitation-data.evaporation)*data.runoff_coeff
variables['withdraw_surf']=data.iloc[:,5:8].sum(axis=1)
variables['gw_irwr']=(data.precipitation-data.evaporation)*(1-data.runoff_coeff)
variables['withdraw_gw']=data.iloc[:,8:11].sum(axis=1)
variables['withdraw_agr']=data.withdraw_gw_agr+data.withdraw_surf_agr
variables['withdraw_ind']=data.withdraw_gw_ind+data.withdraw_surf_ind
variables['access_sanitation_total']=(data.access_sanitation_urban*data.urban_pop_ratio+data.access_sanitation_rural*(1-data.urban_pop_ratio))
variables['deficit_gw_annual']=data.annual_gw_variation*(-1)
variables['deficit_gw_aggregate']=data.aggregate_gw_variation*(-1)
variables['withdraw_total']=np.NaN
variables['withdraw_total']=data[['withdraw_surf_agr','withdraw_surf_ind','withdraw_surf_dom','withdraw_gw_agr','withdraw_gw_ind','withdraw_gw_dom']].sum(axis=1)
variables['agr_withdraw_ratio']=variables['withdraw_agr']/variables['withdraw_total']
variables['access_sanitation_total']=data['access_sanitation_urban']*data['urban_pop_ratio']+data['access_sanitation_rural']*(1-data['urban_pop_ratio'])

#setting negative deficit values equal to zero
variables.loc[variables['deficit_gw_annual'] < 0 ,'deficit_gw_annual']=0
variables.loc[variables['deficit_gw_aggregate'] < 0 ,'deficit_gw_aggregate']=0
variables['withdraw_gw_allowable']=variables.gw_irwr-(variables.deficit_gw_aggregate/17)
variables['withdraw_agr_decrease']=variables.withdraw_gw-variables.withdraw_gw_allowable
variables.loc[variables['withdraw_agr_decrease']<0,'withdraw_agr_decrease']=0
variables['agrwat_lost_ratio']=variables.withdraw_agr_decrease/variables.withdraw_agr


In [6]:
# calculating trade-off ratio between modern irrigation and employment lost

water_saved_perhec=variables['withdraw_agr']/(2*data['land_irrig_tot']-data['land_irrig_modern'])

# wwdpmii= water withdrawal decrease per modern irrigation increase (percent)
wwdpmii=water_saved_perhec*0.01*data['land_irrig_tot']
#wdn=water decrease necessary for compensating the effect of lowering 1 percent modern irrigation
wdn=variables['withdraw_agr_decrease']+wwdpmii
#elsftr=employment lost for tradeoff ratio & mitr=modern irrigation for tradeoff ratio
eltr=wdn*data.agr_employment/variables.withdraw_agr
mitr=data.land_irrig_modern/data.land_irrig_tot-0.01
eln=variables['withdraw_agr_decrease']*data.agr_employment/variables.withdraw_agr
tradeoff_ratio=(eltr-eln)*100/18


In [7]:
#Creating a dataframe for the sub-indicators
#Dimension 1: Resources
sub_indicators=pd.DataFrame()
sub_indicators['rain_coeff_variation']=data['rain_coeff_variation']
sub_indicators['monthly_varicoeff']=data['monthly_varicoeff']
sub_indicators['gw_agdef_gwrwr']=variables.deficit_gw_aggregate/variables.gw_irwr
sub_indicators['gw_andef_withdraw']=variables.deficit_gw_annual/variables.withdraw_gw
sub_indicators['agr_dependency_gw']=data.withdraw_gw_agr/variables.withdraw_agr
sub_indicators['ind_dependency_gw']=data.withdraw_gw_ind/variables.withdraw_ind
sub_indicators['urbanwat_dependency_gw']=data.withdraw_urban_gw/data.produced_urban_wat
sub_indicators['ruralwat_dependency_gw']=data.withdraw_rural_gw/data.produced_rural_wat
sub_indicators['withdraw_surf_ratio']=variables.withdraw_surf/variables.surf_irwr
sub_indicators['withdraw_gw_ratio']=variables.withdraw_gw/variables.gw_irwr
sub_indicators['anomaly_rain']=abs(data['anomaly_rain'])
sub_indicators['anomaly_temp']=data['anomaly_temp']
sub_indicators['irwr_percap']=(variables.irwr*(10**6)/data.population)/(1+data['pop_growth']/100)
#Dimension 2: access

sub_indicators['access_wat_urban']=data['access_wat_urban']
sub_indicators['access_wat_rural']=data['access_wat_rural']
sub_indicators['under_stress_pop']=data['under_stress_pop']
sub_indicators['access_sanitation_urban']=data['access_sanitation_urban']
sub_indicators['access_sanitation_rural']=data['access_sanitation_rural']
sub_indicators['treated_municipal_wastewater']=data.waste_facility_cap/data.daily_produced_waste
sub_indicators['quality_proxy']=data.urban_fam_treatwat/data.total_urban_fam
sub_indicators['damcap_rwr_ratio']=data.dam_cap/variables.surf_irwr

#Dimension 3:Economy
sub_indicators['efficiency_agr']=data.agr_added_value/variables.withdraw_agr
sub_indicators['modern_irrig']=data.land_irrig_modern/data.land_irrig_tot
sub_indicators['employment_lost_agr']=variables.agrwat_lost_ratio*data.agr_employment
sub_indicators['unaccounted_wat_urban']=data['unaccounted_wat_urban']
sub_indicators['unaccounted_wat_rural']=data['unaccounted_wat_rural']
sub_indicators['efficiency_ind']=data.ind_added_value/variables.withdraw_ind

sub_indicators.index=variables['province']
sub_indicators.loc['khuz','damcap_rwr_ratio']=1


In [8]:
#Normalizing Data
normalized=sub_indicators*0
max_allowable_withdraw=variables['withdraw_gw_allowable']/variables['gw_irwr']
max_allowable_withdraw.index=sub_indicators.index

for col in ind_type.columns:
    i=ind_type.columns.get_loc(col)
    if ind_type.loc[2,col]=='b': # The bigger the better indicators
        normalized.loc[sub_indicators[col]>=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]<=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]>ind_type.loc[1,col])& (sub_indicators[col]<ind_type.loc[0,col])
        normalized.loc[cond,col]=(sub_indicators.loc[cond,col]-ind_type.loc[1,col])/(ind_type.loc[0,col]-ind_type.loc[1,col])
    elif ind_type.loc[2,col]=='l':      # The lower the better indicators
        normalized.loc[sub_indicators[col]<=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]>=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]<ind_type.loc[1,col])& (sub_indicators[col]>ind_type.loc[0,col])
        normalized.loc[cond,col]=(ind_type.loc[1,col]-sub_indicators.loc[cond,col])/(ind_type.loc[1,col]-ind_type.loc[0,col])
    elif ind_type.loc[2,col]=='diff':    # GW withdrawal to rwr ratio normalization
        cond1=(sub_indicators[col] > max_allowable_withdraw)
        normalized.loc[cond1,col]=0.01
        cond2=sub_indicators[col]<0.25
        normalized.loc[cond2,col]=1
        cond3=(sub_indicators[col] < max_allowable_withdraw) & (sub_indicators[col]>0.25)
        normalized.loc[cond3,col]=(max_allowable_withdraw.loc[cond3]-sub_indicators.loc[cond3,col])/(max_allowable_withdraw.loc[cond3]-0.25)
        

In [9]:
# dam capacity to surface rwr ratio normalization    
variation_coeff_agg=pd.Series(normalized[['rain_coeff_variation','monthly_varicoeff']].mean(axis=1),index=variables['province'])
variation_coeff_bins=pd.cut(variation_coeff_agg,4,labels=['Q1','Q2','Q3','Q4'])
ideal_damcap=pd.Series(index=variables['province'],dtype='float64') 
ideal_damcap[variation_coeff_bins=='Q1']=1
ideal_damcap[variation_coeff_bins=='Q2']=0.9
ideal_damcap[variation_coeff_bins=='Q3']=0.8
ideal_damcap[variation_coeff_bins=='Q4']=0.7
normalized['damcap_rwr_ratio']=damcap_normalize(sub_indicators['damcap_rwr_ratio'],ideal_damcap)   
normalized.loc['khuz','damcap_rwr_ratio']=0.45

# modifying modern irrigation indicator vlues based on aggregate GW reservoir deficits
gw_deficit_agg=pd.Series(aggregate(normalized[['gw_agdef_gwrwr','gw_andef_withdraw']]),index=variables['province'])
gw_deficit_bins=pd.qcut(gw_deficit_agg,4,labels=['Q1','Q2','Q3','Q4'])
gw_modifier=pd.Series(index=variables['province'],dtype='float64')
gw_modifier[gw_deficit_bins=='Q1']=0.8
gw_modifier[gw_deficit_bins=='Q2']=0.9
gw_modifier[gw_deficit_bins=='Q3']=1
gw_modifier[gw_deficit_bins=='Q4']=1
normalized['modern_irrig']=normalized['modern_irrig'].multiply(gw_modifier)



In [10]:
# reading AHP weights from csv files
eco_weights=pd.read_csv('eco_modified_weights.csv')
acc_weights=pd.read_csv('acc_modified_weights.csv')
res_weights=pd.read_csv('res_modified_weights.csv')
wsi_weights=pd.read_csv('wsi_modified_weights.csv')

eco_weights.drop(columns='Unnamed: 0',inplace=True)
arrayeco=[['E1','E1','E1','E2','E2','E3','E3','E3'],list(eco_weights.iloc[0,:])]
eco_weights.columns=pd.MultiIndex.from_arrays(arrayeco, names=('Aggregation Level', 'Indicator'))
eco_weights.drop(index=0,inplace=True)
eco_weights.index=['Weight']
eco_weights=eco_weights.astype('float')

acc_weights.drop(columns='Unnamed: 0',inplace=True)
arrayacc=[['A1','A1','A1','A2','A2','A2','A3','A3','A3','A3'],list(acc_weights.iloc[0,:])]
acc_weights.columns=pd.MultiIndex.from_arrays(arrayacc, names=('Aggregation Level', 'Indicator'))
acc_weights.drop(index=0,inplace=True)
acc_weights.index=['Weight']
acc_weights=acc_weights.astype('float')

res_weights.drop(columns='Unnamed: 0',inplace=True)
arrayres=[['R1','R1','R2','R2','R3','R3','R3','R3','R4','R4','R5','R5','R5','R6','R6','R7','R7','R7','R7'],list(res_weights.iloc[0,:])]
res_weights.columns=pd.MultiIndex.from_arrays(arrayres, names=('Aggregation Level', 'Indicator'))
res_weights.drop(index=0,inplace=True)
res_weights.index=['Weight']
res_weights=res_weights.astype('float')

wsi_weights.drop(columns='Unnamed: 0',inplace=True)
arraywsi=[['WSI','WSI','WSI'],list(wsi_weights.iloc[0,:])]
wsi_weights.columns=pd.MultiIndex.from_arrays(arraywsi, names=('Aggregation Level', 'Indicator'))
wsi_weights.drop(index=0,inplace=True)
wsi_weights.index=['Weight']
wsi_weights=wsi_weights.astype('float')
ahp_weights=wsi_weights.join(res_weights.join(acc_weights.join(eco_weights,how='outer'),how='outer'),how='outer')
ahp_weights

Aggregation Level      WSI                   R1          R2           R3  \
Indicator         Resource Access Economy   ACV   MCV   Dag   Dan GWDagr   
Weight                 0.5   0.29    0.21  0.72  0.28  0.66  0.34   0.23   

Aggregation Level                ...    A3          E1                 E2  \
Indicator         GWDind GWDurb  ...    DC    SS    AE    MI    EL NRWurb   
Weight              0.09    0.4  ...  0.16  0.18  0.62  0.16  0.22    0.8   

Aggregation Level           E3              
Indicator         NRWrur  Eagr    IE   NRW  
Weight               0.2  0.68  0.15  0.17  

[1 rows x 40 columns]

In [11]:
res_sub_columns=['ACV','MCV','Dag','Dan','GWDagr','GWDind','GWDurb','GWDrur','SWS','GWS','APA','ATA','IRWR']
acc_sub_columns=['WAurb','WArur','USP','SAurb','SArur','TPC','WQ','DC']
eco_sub_columns=['AE','MI','EL','NRWurb','NRWrur','IE']
sub_columns1_list= ['R1','R1','R2','R2','R3','R3','R3','R3','R4','R4','R5','R5','R7']+['A1','A1','A1','A2','A2','A2','A3','A3']+['E1','E1','E1','E2','E2','E3']
sub_columns2_list=res_sub_columns+acc_sub_columns+eco_sub_columns
arraycolumns=[sub_columns1_list]+[sub_columns2_list]
normalized_ahp=pd.DataFrame(normalized).round(decimals=4)
normalized_ahp.columns=pd.MultiIndex.from_arrays(arraycolumns, names=('Aggregation Level', 'Indicator'))
# normalized_ahp.to_excel('Normalized Sub Indicators.xlsx')
normalized_ahp

Aggregation Level      R1              R2              R3                  \
Indicator             ACV     MCV     Dag     Dan  GWDagr  GWDind  GWDurb   
province                                                                    
azsh               0.8302  0.5918  0.7581  0.8357  0.5690  0.4438  0.5411   
azgh               0.7622  0.5862  0.7553  0.9169  0.5936  0.5256  0.5206   
arda               0.9108  0.6729  0.6559  0.5503  0.8440  0.0214  0.7503   
esfe               0.5431  0.2613  0.0100  0.3068  0.3260  0.7052  0.8570   
albo               0.7328  0.4273  0.0638  0.5578  0.3591  0.0523  0.2335   
ilam               0.4564  0.2014  0.9385  1.0000  0.7463  0.4506  0.3795   
bush               0.0418  0.0100  0.8136  0.4482  0.4585  0.4517  0.8414   
tehr               0.7234  0.4456  0.0100  0.7404  0.3993  0.1203  0.4669   
chba               0.6460  0.2508  0.9283  0.8760  0.5809  0.0196  0.0177   
khoj               0.3244  0.1151  0.0100  0.6169  0.1130  0.1163  0.0405   
khor               0.5351  0.3267  0.0100  0.4045  0.1658  0.1962  0.0123   
khosh              0.6422  0.5395  0.2459  0.2910  0.5607  0.0963  0.3170   
khuz               0.5137  0.1223  0.6817  0.0100  0.9337  0.9325  0.5656   
zanj               0.7590  0.5035  0.1686  0.5435  0.3960  0.0376  0.2526   
semn               0.5460  0.3912  0.0100  0.2800  0.3422  0.0979  0.1422   
sisba              0.0100  0.0100  0.2510  0.1569  0.3534  0.3612  0.4284   
fars               0.2478  0.0100  0.6775  0.3974  0.1984  0.3768  0.1423   
qazv               0.7465  0.4194  0.0100  0.2958  0.3146  0.0626  0.0022   
qom                0.6055  0.2619  0.0100  0.8044  0.1391  0.2509  0.8728   
kord               0.7272  0.4100  0.8339  1.0000  0.4820  0.5048  0.5643   
kerm               0.1587  0.0075  0.0100  0.5265  0.0712  0.0066  0.1098   
kersh              0.6735  0.3133  0.8502  1.0000  0.4508  0.1067  0.1172   
kohbu              0.4141  0.0558  0.9919  0.6417  0.7811  0.7307  0.3663   
gole               0.8020  0.7136  0.5698  0.5287  0.3839  0.4072  0.0505   
gilan              0.9163  0.7673  1.0000  1.0000  0.9116  0.7654  0.6081   
lores              0.5991  0.3007  0.9538  0.8680  0.5067  0.2097  0.0100   
mazan              0.9325  0.8216  0.9706  0.9911  0.7740  0.0880  0.1342   
marka              0.6587  0.3235  0.0100  0.3438  0.2517  0.1096  0.4536   
hormo              0.0100  0.0100  0.4899  0.3092  0.0954  0.4394  0.7222   
hame               0.7075  0.3294  0.0100  0.9824  0.1351  0.4641  0.4092   
yazd               0.3781  0.0823  0.0100  0.0100  0.0324  0.0162  0.4686   

Aggregation Level              R4          ...     A2              A3          \
Indicator          GWDrur     SWS     GWS  ...  SArur     TPC      WQ      DC   
province                                   ...                                  
azsh               0.2020  0.3570  0.0284  ...  0.010  0.3865  0.6320  0.4161   
azgh               0.0363  0.3860  0.0100  ...  0.075  0.3893  0.9008  0.7736   
arda               0.4051  0.0100  0.3708  ...  0.030  0.4302  0.5504  0.5807   
esfe               0.5009  0.2615  0.0100  ...  0.040  0.6079  0.6807  0.6042   
albo               0.0270  0.7754  0.0100  ...  0.100  0.1469  0.8116  0.9999   
ilam               0.0100  0.6404  0.7458  ...  0.010  0.4316  0.5489  0.2132   
bush               0.8282  0.0100  0.0407  ...  0.010  0.1186  0.0100  0.6369   
tehr               0.0100  0.0100  0.0100  ...  0.120  0.7635  0.5369  0.4840   
chba               0.0707  1.0000  0.7493  ...  0.010  0.3916  0.9096  0.5129   
khoj               0.0100  0.8677  0.0100  ...  0.090  0.1825  0.2531  0.1958   
khor               0.0384  0.1789  0.0100  ...  0.010  0.0453  0.7829  0.8514   
khosh              0.0436  0.0100  0.0100  ...  0.035  0.1772  0.5233  0.9044   
khuz               0.5978  0.0100  0.3224  ...  0.020  0.0302  0.3006  0.4500   
zanj               0.0100  0.1988  0.0100  ...  0.010  0.2199  0.4553  0.4589   
semn

In [12]:
normalized_lvl2=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
normalized_lvl2.columns=pd.MultiIndex.from_arrays([['R5'],['CV']], names=('Aggregation Level', 'Indicator'))
normalized_lvl2[('R5','CV')]=weighted_addminagg(normalized_ahp.loc[:,'R1'],ahp_weights['R1']).round(decimals=4)
normalized_lvl2[('R6','GD')]=weighted_addminagg(normalized_ahp.loc[:,'R2'],ahp_weights['R2']).round(decimals=4)
normalized_lvl2[('R6','GWD')]=weighted_addminagg(normalized_ahp.loc[:,'R3'],ahp_weights['R3']).round(decimals=4)

## Assessing water security using a mix of weighted_sum and minimum aggregation rules

#  calculating indicators 
indicators_weighted=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
indicators_weighted.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_weighted[('Resource','IRWR')]=normalized_ahp.loc[:,('R7','IRWR')]
indicators_weighted[('Resource','CC')]=weighted_addminagg(normalized_lvl2['R5'].merge(normalized_ahp['R5'], left_index=True, right_index=True,how='outer'),ahp_weights['R5']).round(decimals=4)
indicators_weighted[('Resource','WS')]=weighted_addminagg(normalized_ahp.loc[:,'R4'],ahp_weights['R4']).round(decimals=4)
# if a province has low groundwater reservoir deficit then high dependency on groundwater is considered okay i.e. if GD> 0.8 then GWD not involved in aggregation
indicators_weighted.loc[normalized_lvl2[('R6','GD')]<0.8,('Resource','GW')]=weighted_addminagg(normalized_lvl2.loc[:,'R6'],ahp_weights['R6']).round(decimals=4)
indicators_weighted.loc[normalized_lvl2[('R6','GD')]>0.8,('Resource','GW')]=normalized_lvl2.loc[:,('R6','GD')]
indicators_weighted[('Access','DW')]=weighted_addminagg(normalized_ahp.loc[:,'A1'],ahp_weights['A1']).round(decimals=4)
indicators_weighted[('Access','WQ')]=normalized_ahp.loc[:,('A3','WQ')]
indicators_weighted[('Access','DC')]=normalized_ahp.loc[:,('A3','DC')]
indicators_weighted[('Access','SS')]=weighted_addminagg(normalized_ahp.loc[:,'A2'],ahp_weights['A2']).round(decimals=4)
indicators_weighted[('Economy','Eagr')]=weighted_addminagg(normalized_ahp.loc[:,'E1'],ahp_weights['E1']).round(decimals=4)
indicators_weighted[('Economy','IE')]=normalized_ahp.loc[:,('E3','IE')]
indicators_weighted[('Economy','NRW')]=weighted_addminagg(normalized_ahp.loc[:,'E2'],ahp_weights['E2']).round(decimals=4)

#  calculating dimensions & WSI
Dimensions_addminagg=pd.DataFrame(index=sub_indicators.index)
Dimensions_addminagg['Resource']=weighted_addminagg(indicators_weighted[('Resource')],ahp_weights['R7']).round(decimals=4)
Dimensions_addminagg['Access']=weighted_addminagg(indicators_weighted[('Access')],ahp_weights['A3']).round(decimals=4)
Dimensions_addminagg['Economy']=weighted_addminagg(indicators_weighted[('Economy')],ahp_weights['E3']).round(decimals=4)
Dimensions_addminagg['WSI']=weighted_addminagg(Dimensions_addminagg,ahp_weights['WSI']).round(decimals=4)
Dimensions_addminagg.sort_values('Access',ascending=False)
indicators_weighted['Access'].to_excel('Access Dimension.xlsx')

In [13]:
# correlation analysis
Res_vs_subs=normalized_ahp.iloc[:,:13].join(Dimensions_addminagg['Resource'])
Acc_vs_subs=normalized_ahp.iloc[:,13:21].join(Dimensions_addminagg['Access'])
Eco_vs_subs=normalized_ahp.iloc[:,21:].join(Dimensions_addminagg['Economy'])
Res_corr=Res_vs_subs.corr(method='pearson')['Resource']
Acc_corr=Acc_vs_subs.corr(method='kendall')['Access']
Eco_corr=Eco_vs_subs.corr(method='kendall')['Economy']
Eco_corr

D:\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


(E1, AE)        0.359140
(E1, MI)        0.277419
(E1, EL)        0.150904
(E2, NRWurb)    0.314317
(E2, NRWrur)    0.058316
(E3, IE)        0.681720
Economy         1.000000
Name: Economy, dtype: float64

In [14]:
# writer = pd.ExcelWriter('Normalized Indicators.xlsx', engine='xlsxwriter')
# indicators_weighted.to_excel(writer,sheet_name='Indicators')
# normalized_lvl2.to_excel(writer,sheet_name='Level 2 Sub_Indicators')
# writer.save()

In [15]:
## Assessing water security using weighted_sum aggregation 
normalized_lvl2=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
normalized_lvl2.columns=pd.MultiIndex.from_arrays([['R5'],['CV']], names=('Aggregation Level', 'Indicator'))
normalized_lvl2[('R5','CV')]=weighted_sum(normalized_ahp.loc[:,'R1'],ahp_weights['R1']).round(decimals=4)
normalized_lvl2[('R6','GD')]=weighted_sum(normalized_ahp.loc[:,'R2'],ahp_weights['R2']).round(decimals=4)
normalized_lvl2[('R6','GWD')]=weighted_sum(normalized_ahp.loc[:,'R3'],ahp_weights['R3']).round(decimals=4)

indicators_weighted=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
indicators_weighted.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_weighted[('Resource','IRWR')]=normalized_ahp.loc[:,('R7','IRWR')]
indicators_weighted[('Resource','CC')]=weighted_sum(normalized_lvl2['R5'].merge(normalized_ahp['R5'], left_index=True, right_index=True,how='outer'),ahp_weights['R5'])
indicators_weighted[('Resource','WS')]=weighted_sum(normalized_ahp.loc[:,'R4'],ahp_weights['R4'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]<0.8,('Resource','GW')]=weighted_sum(normalized_lvl2.loc[:,'R6'],ahp_weights['R6'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]>0.8,('Resource','GW')]=normalized_lvl2.loc[:,('R6','GD')]
indicators_weighted[('Access','DW')]=weighted_sum(normalized_ahp.loc[:,'A1'],ahp_weights['A1'])
indicators_weighted[('Access','WQ')]=normalized_ahp.loc[:,('A3','WQ')]
indicators_weighted[('Access','DC')]=normalized_ahp.loc[:,('A3','DC')]
indicators_weighted[('Access','SS')]=weighted_sum(normalized_ahp.loc[:,'A2'],ahp_weights['A2'])
indicators_weighted[('Economy','Eagr')]=weighted_sum(normalized_ahp.loc[:,'E1'],ahp_weights['E1'])
indicators_weighted[('Economy','IE')]=normalized_ahp.loc[:,('E3','IE')]
indicators_weighted[('Economy','NRW')]=weighted_sum(normalized_ahp.loc[:,'E2'],ahp_weights['E2'])

#  calculating dimensions & WSI
Dimensions_weightedsum=pd.DataFrame(index=sub_indicators.index)
Dimensions_weightedsum['Resource']=weighted_sum(indicators_weighted[('Resource')],ahp_weights['R7'])
Dimensions_weightedsum['Access']=weighted_sum(indicators_weighted[('Access')],ahp_weights['A3'])
Dimensions_weightedsum['Economy']=weighted_sum(indicators_weighted[('Economy')],ahp_weights['E3'])
Dimensions_weightedsum['WSI']=weighted_sum(Dimensions_weightedsum,ahp_weights['WSI']).round(decimals=4)
Dimensions_weightedsum.sort_values('WSI',ascending=False)


,Resource,Access,Economy,WSI
province,,,,
lores,0.916019,0.609229,0.588077,0.7582
mazan,0.828225,0.524735,0.719407,0.7174
gilan,0.938836,0.585898,0.369598,0.7169
kord,0.850139,0.619252,0.528967,0.7157
chba,0.863598,0.669958,0.397265,0.7095
kohbu,0.880330,0.550208,0.489675,0.7026
ilam,0.835903,0.639993,0.401982,0.6880
kersh,0.770479,0.508570,0.407824,0.6184
azgh,0.554183,0.670203,0.379276,0.5511


In [16]:
# writer = pd.ExcelWriter('WSI_weighted.xlsx', engine='xlsxwriter')
# WSI.to_excel(writer, sheet_name = 'تجمیع ترکیبی')
# WSI2.to_excel(writer, sheet_name = 'تجمیع حسابی')
# writer.save()

In [17]:
## Assessing water security using a weighted_geomean aggregation rule
normalized_lvl2=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
normalized_lvl2.columns=pd.MultiIndex.from_arrays([['R5'],['CV']], names=('Aggregation Level', 'Indicator'))
normalized_lvl2[('R5','CV')]=weighted_geomean(normalized_ahp.loc[:,'R1'],ahp_weights['R1']).round(decimals=4)
normalized_lvl2[('R6','GD')]=weighted_geomean(normalized_ahp.loc[:,'R2'],ahp_weights['R2']).round(decimals=4)
normalized_lvl2[('R6','GWD')]=weighted_geomean(normalized_ahp.loc[:,'R3'],ahp_weights['R3']).round(decimals=4)

indicators_weighted=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
indicators_weighted.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_weighted[('Resource','IRWR')]=normalized_ahp.loc[:,('R7','IRWR')]
indicators_weighted[('Resource','CC')]=weighted_geomean(normalized_lvl2['R5'].merge(normalized_ahp['R5'], left_index=True, right_index=True,how='outer'),ahp_weights['R5'])
indicators_weighted[('Resource','WS')]=weighted_geomean(normalized_ahp.loc[:,'R4'],ahp_weights['R4'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]<0.8,('Resource','GW')]=weighted_geomean(normalized_lvl2.loc[:,'R6'],ahp_weights['R6'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]>0.8,('Resource','GW')]=normalized_lvl2.loc[:,('R6','GD')]
indicators_weighted[('Access','DW')]=weighted_geomean(normalized_ahp.loc[:,'A1'],ahp_weights['A1'])
indicators_weighted[('Access','WQ')]=normalized_ahp.loc[:,('A3','WQ')]
indicators_weighted[('Access','DC')]=normalized_ahp.loc[:,('A3','DC')]
indicators_weighted[('Access','SS')]=weighted_geomean(normalized_ahp.loc[:,'A2'],ahp_weights['A2'])
indicators_weighted[('Economy','Eagr')]=weighted_geomean(normalized_ahp.loc[:,'E1'],ahp_weights['E1'])
indicators_weighted[('Economy','IE')]=normalized_ahp.loc[:,('E3','IE')]
indicators_weighted[('Economy','NRW')]=weighted_geomean(normalized_ahp.loc[:,'E2'],ahp_weights['E2'])

#  calculating dimensions & WSI
Dimensions_weightedgeo=pd.DataFrame(index=sub_indicators.index)
Dimensions_weightedgeo['Resource']=weighted_geomean(indicators_weighted[('Resource')],ahp_weights['R7'])
Dimensions_weightedgeo['Access']=weighted_geomean(indicators_weighted[('Access')],ahp_weights['A3'])
Dimensions_weightedgeo['Economy']=weighted_geomean(indicators_weighted[('Economy')],ahp_weights['E3'])
Dimensions_weightedgeo['WSI']=weighted_geomean(Dimensions_weightedgeo,ahp_weights['WSI']).round(decimals=4)
Dimensions_weightedgeo.sort_values('WSI',ascending=False)

,Resource,Access,Economy,WSI
province,,,,
lores,0.904152,0.330238,0.480961,0.5913
gilan,0.927281,0.509926,0.246514,0.5903
mazan,0.794204,0.337737,0.616586,0.5877
kord,0.827680,0.560777,0.274369,0.5863
chba,0.837131,0.602035,0.240812,0.5857
kohbu,0.774409,0.449690,0.332185,0.5538
ilam,0.783427,0.532195,0.232158,0.5425
kersh,0.713931,0.361844,0.287458,0.4843
arda,0.307823,0.469515,0.405742,0.3687


In [18]:
## Assessing water security using a mix of weighted_sum and weighted_geomean rules
normalized_lvl2=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
normalized_lvl2.columns=pd.MultiIndex.from_arrays([['R5'],['CV']], names=('Aggregation Level', 'Indicator'))
normalized_lvl2[('R5','CV')]=weighted_addgeo(normalized_ahp.loc[:,'R1'],ahp_weights['R1']).round(decimals=4)
normalized_lvl2[('R6','GD')]=weighted_addgeo(normalized_ahp.loc[:,'R2'],ahp_weights['R2']).round(decimals=4)
normalized_lvl2[('R6','GWD')]=weighted_addgeo(normalized_ahp.loc[:,'R3'],ahp_weights['R3']).round(decimals=4)

indicators_weighted=pd.DataFrame(normalized_ahp[('R4','SWS')]*0,index=sub_indicators.index)
indicators_weighted.columns=pd.MultiIndex.from_arrays([['Resource'],['IRWR']], names=('Dimension', 'Indicator'))
indicators_weighted[('Resource','IRWR')]=normalized_ahp.loc[:,('R7','IRWR')]
indicators_weighted[('Resource','CC')]=weighted_addgeo(normalized_lvl2['R5'].merge(normalized_ahp['R5'], left_index=True, right_index=True,how='outer'),ahp_weights['R5'])
indicators_weighted[('Resource','WS')]=weighted_addgeo(normalized_ahp.loc[:,'R4'],ahp_weights['R4'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]<0.8,('Resource','GW')]=weighted_addgeo(normalized_lvl2.loc[:,'R6'],ahp_weights['R6'])
indicators_weighted.loc[normalized_lvl2[('R6','GD')]>0.8,('Resource','GW')]=normalized_lvl2.loc[:,('R6','GD')]
indicators_weighted[('Access','DW')]=weighted_addgeo(normalized_ahp.loc[:,'A1'],ahp_weights['A1'])
indicators_weighted[('Access','WQ')]=normalized_ahp.loc[:,('A3','WQ')]
indicators_weighted[('Access','DC')]=normalized_ahp.loc[:,('A3','DC')]
indicators_weighted[('Access','SS')]=weighted_addgeo(normalized_ahp.loc[:,'A2'],ahp_weights['A2'])
indicators_weighted[('Economy','Eagr')]=weighted_addgeo(normalized_ahp.loc[:,'E1'],ahp_weights['E1'])
indicators_weighted[('Economy','IE')]=normalized_ahp.loc[:,('E3','IE')]
indicators_weighted[('Economy','NRW')]=weighted_addgeo(normalized_ahp.loc[:,'E2'],ahp_weights['E2'])

#  calculating dimensions & WSI
Dimensions_addgeo=pd.DataFrame(index=sub_indicators.index)
Dimensions_addgeo['Resource']=weighted_addgeo(indicators_weighted[('Resource')],ahp_weights['R7'])
Dimensions_addgeo['Access']=weighted_addgeo(indicators_weighted[('Access')],ahp_weights['A3'])
Dimensions_addgeo['Economy']=weighted_addgeo(indicators_weighted[('Economy')],ahp_weights['E3'])
Dimensions_addgeo['WSI']=weighted_addgeo(Dimensions_addgeo,ahp_weights['WSI']).round(decimals=4)
Dimensions_addgeo.sort_values('WSI',ascending=False)

,Resource,Access,Economy,WSI
province,,,,
lores,0.910298,0.469117,0.531917,0.6869
gilan,0.933205,0.554249,0.303492,0.6624
kord,0.839461,0.589723,0.399211,0.6614
mazan,0.811367,0.432517,0.664456,0.6595
chba,0.850782,0.640490,0.313657,0.6562
kohbu,0.833355,0.501421,0.409350,0.6338
ilam,0.812406,0.588914,0.312494,0.6241
kersh,0.743764,0.432520,0.351890,0.5572
azgh,0.448883,0.649148,0.346578,0.4793


In [19]:
# writer = pd.ExcelWriter('Dimensions & WSI.xlsx', engine='xlsxwriter')
# Dimensions_addminagg.to_excel(writer,sheet_name='Mean-Min')
# Dimensions_weightedsum.to_excel(writer,sheet_name='Mean')
# Dimensions_weightedgeo.to_excel(writer,sheet_name='Geo')
# Dimensions_addgeo.to_excel(writer,sheet_name='Mean-Geo')
# writer.save()